# I. Importation des bibliothèques

In [1]:
%%capture
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib unidecode

In [2]:
%%capture
import warnings

warnings.filterwarnings("ignore")

import os
from pathlib import Path

import pandas as pd
from openhexa.sdk import workspace
from unidecode import unidecode

pd.set_option("display.max_columns", None)

try:
    import openpyxl as pyxl
    from efc.interfaces.iopenpyxl import OpenpyxlInterface
    from fuzzywuzzy import fuzz, process
except ImportError or ModuleNotFoundError:
    !pip install --quiet fuzzywuzzy python-Levenshtein excel-formulas-calculator
    import openpyxl as pyxl
    from efc.interfaces.iopenpyxl import OpenpyxlInterface

## 1. Rafraîchissement des bibliothèques

In [3]:
%%capture
# Ajout du chemin d'accès pour l'importation des bibliothèques
os.chdir(Path(workspace.files_path, "Fichier Suivi de Stock/code/pipelines"))

from importlib import reload

import compute_indicators
import export_file_to_google_drive as ggdrive
import generate_stock_tracking_file as gstf

# Importation des réquêtes sql
from compute_indicators.queries import *
from database_operations import stock_sync_manager

# Reload modules
reload(compute_indicators)
reload(stock_sync_manager)
reload(gstf)
reload(ggdrive)

# II. Définition des paramètres

## 1. Variables requises pour la conception du fichier de suivi de stock

1. **Mois de création du rapport** : Si le processus est bien défini, cette valeur peut être automatisée. Sinon, elle sera fournie en tant qu'entrée du pipeline.

2. **Programme** : Le programme concerné pour lequel le fichier de suivi de stock est généré.

3. **Fichier État Mensuel** : Fichier transmis mensuellement par la N-PSP (source : SAGE X3).

4. **Fichier Plan d'Approvisionnement** : Utilisé pour certains calculs, notamment dans la feuille « Prévision ». Ce fichier est généralement récupéré depuis la source QAT.

5. **Fichier de mappage des produits** : Fichier assurant la correspondance entre les produits de SAGE X3 et ceux de QAT.

In [4]:
month_report, year_report, programme, fp_etat_mensuel, fp_plan_approv, fp_map_prod = (
    "Mars",
    2025,
    "PNLP",
    "Etat du stock et de distribution PNLP fin Mars 2025.xlsx",
    "Détails de l`envoiMar. 2025 ~ Dec. 2026.csv",
    "Mapping QAT_SAGEX3.xlsx",
)

In [5]:
# Parameters
month_report = "Mars"
year_report = 2025
programme = "PNLS"
fp_etat_mensuel = "Etat du stock et de distribution PNLS fin Mars 2025.xlsx"
fp_plan_approv = "Mars 2025"
fp_map_prod = "Mapping QAT_SAGEX3.xlsx"


In [6]:
month_export, date_report = month_report, compute_indicators.utils.format_date(month_report, year_report)

date_report_prec = (pd.to_datetime(date_report).replace(day=1) - pd.offsets.MonthBegin()).strftime('%Y-%m-%d')

## 2. Test pour s'assurer que cette date n'existe pas déjà dans la base de données

In [7]:
stock_sync_manager.initialize_database_connection()

schema_name = "suivi_stock"

Connexion à la base de données établie avec succès


In [8]:
# Ici il faudrait également s'assurer que le mois précédent est bien présent à l'intérieur de la base de données au cas où on arriverait à se tromper sur le mois de conception du fichier
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select * 
    from {schema_name}.stock_track st 
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report_prec}'
    limit 2
    """
)

assert (
    df_.shape[0] != 0
), f"Le mois précédent {date_report_prec} n'est pas présent dans la base de données locale êtes-vous sûre d'avoir choisir le bon mois de conception du fichier."

del df_

# III.📥Importation des Données
  
L'utilisateur doit veiller à ce que les fichiers respectent le format attendu et soient placés dans les répertoires dédiés avant de procéder au traitement.

## 📌1. Importation du fichier `Etat de Stock Mensuel`

- **Format requis :** Assurez-vous que le fichier respecte le template standard défini.
- **Emplacement du fichier :** Le fichier doit être placé dans le répertoire dédié :  
  **`Fichier Suivi de Stock/data/<programme>/Etat de Stock Mensuel`**
  
- **En cas d'erreur :**  
  - Vérifiez que le fichier est bien présent dans le répertoire.  
  - Assurez-vous que toutes les colonnes requises sont bien renseignées.  
  - Contrôlez que le fichier est bien accessible et non corrompu.

In [9]:
fp_etat_mensuel = (
    Path(workspace.files_path)
    / f"Fichier Suivi de Stock/data/{programme}/Etat de Stock Mensuel"
    / Path(fp_etat_mensuel).name
)

sheetnames = pd.ExcelFile(fp_etat_mensuel).sheet_names

## 📌2. Importation de la feuille `Etat de stock de la NPSP`

In [10]:
sheet_stock_npsp = compute_indicators.utils.check_if_sheet_name_in_file('Etat de stock', sheetnames)

assert sheet_stock_npsp is not None, print(f"La feuille `Etat de stock` n'est pas dans la liste {sheetnames} du classeur excel")

df_etat_stock_npsp = pd.read_excel(fp_etat_mensuel, sheet_name=sheet_stock_npsp, skiprows=4)
df_etat_stock_npsp.columns = df_etat_stock_npsp.columns.str.strip()

del sheet_stock_npsp

df_etat_stock_npsp = df_etat_stock_npsp.loc[df_etat_stock_npsp["Nouveau code"].notna()]

df_etat_stock_npsp = compute_indicators.file_utils.process_etat_stock_npsp(df_etat_stock_npsp, date_report, programme)

df_etat_stock_npsp.head(2)

,date_report,code_produit,designation,contenance,dmm,traceurs,stock_theorique_bke,stock_theorique_abj,stock_theorique_central,stock_theorique_fin_mois,msd,statut_stock,programme
2,2025-03-01,3050188.0,COTRIMOXAZOLE 120 (TRIMETHOPRIME 20+SULFAMETHO...,BOITE/1000,820.166667,0.0,77.0,280.0,0.0,357.0,0.435277,Sous-stock,PNLS
3,2025-03-01,3050191.0,COTRIMOXAZOLE 960 (TRIMETHOPRIME + SULFAMET. 1...,COMPRIME,2340928.166667,1.0,4212870.0,7247159.0,0.0,11460029.0,4.895506,Bon,PNLS


## 📌3. Importation de la feuille `Stock detaille`

In [11]:
sheet_stock_detaille = compute_indicators.utils.check_if_sheet_name_in_file('Stock detaille', sheetnames)

assert sheet_stock_detaille is not None, print(f"La feuille `Stock detaille` n'est pas dans la liste {sheetnames} du classeur excel")

df_stock_detaille = pd.read_excel(fp_etat_mensuel, sheet_name=sheet_stock_detaille)
df_stock_detaille.columns = df_stock_detaille.columns.str.strip()

max_date_year = pd.Timestamp.max.year

df_stock_detaille["Date limite de consommation"] = df_stock_detaille["Date limite de consommation"].apply(
    lambda x: x if x.year<max_date_year else x.replace(year=max_date_year-1)
)

del sheet_stock_detaille, max_date_year

df_stock_detaille.head(2)

,Code produit,Désignation,Emplacement,Date limite de consommation,Numéro Lot,Sous lot (Programme),Qté \nPhysique,Qté \nlivrable,Unit,Site,Dépôt,Sous lot,Site.1,Bailleur,Statut
0,4040095,COBAS C111 SAMPLE CUPS sach/250 (C 111/C 311),01A01A01,2099-01-01,NEANT,PNLS,1650,0,SACH,PG01,PGWIB,PNLS,PG01,BAILLEURX,Q
1,3100039,RALTEGRAVIR 400 mg comp. BTE/60,01A01A02,2026-11-01,4524,PNLS,780,540,BTE,PG01,PGWIB,PNLS,PG01,LHSPLA,A


## 📌4. Importation de la feuille `Distribution X3`

In [12]:
sheet_distribution_x3 = compute_indicators.utils.check_if_sheet_name_in_file('Distribution', sheetnames)

assert sheet_distribution_x3 is not None, print(f"La feuille `Distribution X3` n'est pas dans la liste {sheetnames} du classeur excel")

df_distribution = pd.read_excel(fp_etat_mensuel, sheet_name=sheet_distribution_x3)
df_distribution.columns = df_distribution.columns.str.strip()

del sheet_distribution_x3

df_distribution.head(2)

,Programme,Date de livraison,No commande,No livraison,Client,Raison sociale,Article,Désignation,Unité vente,Qté commandée,Qté livrée
0,PNLS,2025-03-04,VAL01-PSG-2501-1804,BL01-PSG-2503-0223,60300244,FSU COM ABOBO SAGBE,3120045,PRESERVATIF MASCULIN PQT/100 PQT -,PQT,162,162
1,PNLS,2025-03-04,VAL01-PSG-2501-1804,BL01-PSG-2503-0223,60300244,FSU COM ABOBO SAGBE,3100042,TENOFOVIR / LAMIVUDINE 300 mg / 300 mg comp. B...,BTE,23,23


## 📌5. Importation de la feuille `Receptions`

In [13]:
sheet_reception = compute_indicators.utils.check_if_sheet_name_in_file('Receptions', sheetnames)

assert sheet_reception is not None, print(f"La feuille `Receptions` n'est pas dans la liste {sheetnames} du classeur excel")

df_receptions = pd.read_excel(fp_etat_mensuel, sheet_name=sheet_reception)
df_receptions.columns = df_receptions.columns.str.strip()

del sheet_reception

df_receptions.head(2)

,Programme,Bailleur (Funding),Nouveau code,Code,Désignation NPSP,Date de réception effective,Date effective de dépotage et/ou expertise,Quantité réceptionnée,Date d'entrée en machine
0,PNLS,FONDS MONDIAL,4040328,ND,WONDFO HIV 1/2 TEST KIT/25,2025-02-17,2025-02-19,14080,2025-03-07
1,PNLS,FONDS MONDIAL,4040328,ND,WONDFO HIV 1/2 TEST KIT/25,2025-02-17,2025-02-19,9660,2025-03-07


## 📌6. Importation de la feuille `PPI`

In [14]:
sheet_ppi = compute_indicators.utils.check_if_sheet_name_in_file('PPI', sheetnames)

assert sheet_ppi is not None, print(f"La feuille `PPI` n'est pas dans la liste {sheetnames} du classeur excel")

df_ppi = pd.read_excel(fp_etat_mensuel, sheet_name=sheet_ppi, skiprows=2)
df_ppi.columns = df_ppi.columns.str.strip()

del sheet_ppi

df_ppi.head(2)

,Code Produit,Nom Produit,Unite,Numéro Lot,Date Peremption,Quantité
0,3100023,EFAVIRENZ/LAMIVUDINE/TENOFOVIR 400/300/300 mg ...,BTE,22075712,2025-04-01,2552
1,3100023,EFAVIRENZ/LAMIVUDINE/TENOFOVIR 400/300/300 mg ...,BTE,22075712,2025-04-01,9157


## 📌7. Importation de la feuille  `Prélèvement`

In [15]:
sheet_prelev = compute_indicators.utils.check_if_sheet_name_in_file('Prelèvement CQ', sheetnames)

assert sheet_prelev is not None, print(f"La feuille `Prelèvement CQ` n'est pas dans la liste {sheetnames} du classeur excel")

df_prelevement = pd.read_excel(fp_etat_mensuel, sheet_name=sheet_prelev, skiprows=2)
df_prelevement.columns = df_prelevement.columns.str.strip()

del sheet_prelev

df_prelevement.head(2)

,Code Produit,Nom Produit,Unite,Numéro Lot,Date Peremption,Quantité


## 📌8. Importation du fichier de `mapping des produits QAT et SAGE X3`

- **Format requis :** Respectez la structure du fichier.  
- **Emplacement du fichier :** Le fichier doit être placé dans le répertoire suivant :  
  **`Fichier Suivi de Stock/data/Mapping produits QAT SAGE X3`**  
- **En cas d'erreur :**
  - Vérifiez que le fichier est bien présent dans le répertoire.  
  - Assurez-vous que les données respectent le format attendu. 
  - Vérifiez l'intégrité du fichier et assurez-vous et corriger l'anomalie.

In [16]:
fp_map_prod = (
    Path(workspace.files_path)
    / "Fichier Suivi de Stock/data/Mapping produits QAT SAGE X3"
    / Path(fp_map_prod).name
)

In [17]:
assert fp_map_prod.exists(), "Le fichier de mapping des produits n'a pas été retrouvé dans le dossier dédié"

## 📌9. Importation du `Plan d'approvisionnement`

- **Format requis :** Respectez la structure du fichier définie pour l’importation peut conserver le fichier brut extrait de QAT.  
- **Emplacement du fichier :** Le fichier doit être placé dans le répertoire suivant :  
  **`Fichier Suivi de Stock/data/<programme>/Plan d'Approvisionnement`**  
- **En cas d'erreur :**
  - Vérifiez que le fichier est bien présent dans le répertoire.  
  - Assurez-vous que les données respectent le format attendu.
  - Vérifiez l'intégrité du fichier et assurez-vous et corriger l'anomalie.

In [18]:
fp_plan_approv = (
    Path(workspace.files_path)
    / f"Fichier Suivi de Stock/data/{programme}/Plan d'Approvisionnement"
    / Path(fp_plan_approv)
)

In [19]:
df_plan_approv = compute_indicators.file_utils.process_pa_files(fp_plan_approv, fp_map_prod, programme)

df_plan_approv["facteur_de_conversion_qat_sage"] = df_plan_approv["facteur_de_conversion_qat_sage"].apply(
    lambda x: x if not pd.isna(x) else 1
)

df_plan_approv.head(2)

,ID de produit QAT,Produits,ID de l`envoi QAT,Commande d`urgence,Commande PGI,Approvisionnement local,N° de commande de l`agent d`approvisionnement,Agent d`approvisionnement,Source de financement,Budget,Status,Quantite,DATE,Cout des Produits,Couts du fret,Couts totaux,Notes,Standard product code,cout_unitaire_moyen_qat,facteur_de_conversion_qat_sage,acronym,code_and_date_concate
0,3067,Female Condom (Nitrile) Lubricated 17 cm 100...,173896,false,false,false,,PPM-GF,GFATM,GF_2024,Planifié,200.000,2025-12-31,100000.00,10000.0,110000.00,,3120042.0,506.923077,1.0,NaN,3120042_2025-12-31
1,4182,Male Condom (Latex) Lubricated No Logo 53 mm...,173899,false,false,false,,PPM-GF,GFATM,GF_2024,Soumis,4147.584,2025-07-31,401776.46,40181.7,441958.16,,3120044.0,96.632703,1.0,CONDOM M 144,3120044_2025-07-31


# IV.📊Calcul des Indicateurs

## 📌1. `Etat de Stock (1/2)`

In [20]:
stock_sync_manager.initialize_database_connection()

schema_name = "suivi_stock"

df_etat_stock = stock_sync_manager.get_table_data(
    query=QUERY_ETAT_STOCK.format(
        schema_name=schema_name, programme=programme, date_report_prec=date_report_prec
    )
).drop_duplicates()

df_etat_stock.head(2)

Connexion à la base de données établie avec succès


,id_dim_produit_stock_track_pk,code_produit,ancien_code,categorie,designation,type_produit,unit_niveau_central,unit_niveau_peripherique,facteur_de_conversion,designation_acronym,code_qat,programme,cout_unitaire_moyen_qat,facteur_de_conversion_qat_sage,stock_theorique_mois_precedent
0,40,3100001,AR33198,MEDICAMENTS ARV,ABACAVIR / LAMIVUDINE 120/60 mg comp disp. BTE/30,Traceur,BTE/30,COMPRIME,30,ABC/3TC 120/60MG CP BTE/30,1073.0,PNLS,3.316414,1.0,50058.0
1,41,3100003,AR33196,MEDICAMENTS ARV,ABACAVIR / LAMIVUDINE 600 mg / 300 mg comp. BT...,Non traceur,BTE/30,COMPRIME,30,ABC/3TC 120/60MG CP BTE/30,1082.0,PNLS,9.000000,1.0,34968.0


In [21]:
%%time
df_etat_stock = compute_indicators.annexe_1.get_etat_stock_current_month(
    df_etat_stock.copy(),
    df_stock_detaille.copy(),
    df_distribution.copy(),
    df_ppi.copy(),
    df_prelevement.copy(),
    df_receptions.copy(),
    date_report
)

,id_dim_produit_stock_track_pk,code_produit,ancien_code,categorie,designation,type_produit,unit_niveau_central,unit_niveau_peripherique,facteur_de_conversion,designation_acronym,code_qat,programme,cout_unitaire_moyen_qat,facteur_de_conversion_qat_sage,stock_theorique_mois_precedent,Distribution effectuée,Quantité reçue entrée en stock,Quantité de PPI,Quantité prélévée en Contrôle Qualité (CQ),Ajustement de stock,Stock Théorique Final SAGE,Stock Théorique Final Attendu,ECARTS,Justification des écarts,Diligences,Dilig. Choisie
0,40,3100001,AR33198,MEDICAMENTS ARV,ABACAVIR / LAMIVUDINE 120/60 mg comp disp. BTE/30,Traceur,BTE/30,COMPRIME,30,ABC/3TC 120/60MG CP BTE/30,1073.0,PNLS,3.316414,1.0,50058.0,4643,0,0,0,None,45532,45415.0,117.0,None,None,None
1,41,3100003,AR33196,MEDICAMENTS ARV,ABACAVIR / LAMIVUDINE 600 mg / 300 mg comp. BT...,Non traceur,BTE/30,COMPRIME,30,ABC/3TC 120/60MG CP BTE/30,1082.0,PNLS,9.000000,1.0,34968.0,1391,0,0,0,None,34201,33577.0,624.0,None,None,None
2,42,3100004,AR33195,MEDICAMENTS ARV,ABACAVIR 300 mg comp. BTE/60,Non traceur,BTE/60,COMPRIME,60,None,1069.0,PNLS,10.965459,1.0,0.0,135,0,0,0,None,4145,-135.0,4280.0,None,None,None


CPU times: user 650 ms, sys: 0 ns, total: 650 ms
Wall time: 650 ms


## 📌2.`Distributions et Consommations` - `Annexe 1`

### 2.1.`Distributions`

In [22]:
%%time
df_dmm_curent, df_dmm_histo = compute_indicators.annexe_1.get_dmm_current_month(
    df_etat_stock.copy(),
    programme,
    date_report,
    stock_sync_manager.civ_engine
)

CPU times: user 298 ms, sys: 3.65 ms, total: 302 ms
Wall time: 351 ms


### 2.2.`Consommations`

In [23]:
# Pour avoir la CMM du mois courant une recherche est d'abord faite sur la feuille StockParRegion provenant du RapportFeedback
eomonth = (pd.to_datetime(date_report) + pd.offsets.MonthEnd(0)).strftime("%Y-%m-%d")

df_stock_prog_nat = stock_sync_manager.get_table_data(
    query = QUERY_ETAT_STOCK_PROGRAMME.format(eomonth=eomonth, programme=programme)
)

df_stock_prog_nat["Code_produit"] = df_stock_prog_nat["Code_produit"].astype(int)

df_stock_prog_nat.head(3)

,Programme,MSD,STATUT,CONSO,SDU,CMM,dispo_globale,dispo_globale_cible,dispo_traceur,dispo_traceur_cible,date_report,id_produit_fk,Code_region,region_order,statut_pourcentage,id_produit_pk,Code_produit,Produit_designation,Unit_rapportage,Categorie_produit,Categorie_du_produit,Code_sous_prog
0,PNLS,"3,7",BIEN STOCKE,156255,588198,158253.50,0.002797,0.003441,0.003649,0.003846,2025-03-31,97,NAT,None,0.004049,97,3100016,DOLUTEGRAVIR 10 mg comp. dispersible BTE/90 BT...,COMPRIME,Produit traceur,ARV-ENFANT,PNLS-1
1,PNLS,"2,7",BIEN STOCKE,630487,2218676,814590.50,0.002797,0.003441,0.003649,0.003846,2025-03-31,161,NAT,None,0.004049,161,3120044,PRESERVATIF MASCULIN BTE/144 BTE -,PRESERVATIF,Produit non traceur,AUTRE,PNLS-1
2,PNLS,"4,0",BIEN STOCKE,222084,1104291,278226.25,0.002797,0.003441,0.003649,0.003846,2025-03-31,95,NAT,None,0.004049,95,3100013,DOLUTEGRAVIR /LAMIVUDINE/TENOFOVIR 50/300/300 ...,COMPRIME,Produit non traceur,MEDICAMENTS ET INTRANTS,PNLS-1


In [24]:
%%time
df_cmm_curent, df_cmm_histo = compute_indicators.annexe_1.get_cmm_current_month(
    df_etat_stock.copy(),
    df_stock_prog_nat.copy(),
    programme,
    date_report,
    stock_sync_manager.civ_engine,
)

CPU times: user 307 ms, sys: 889 μs, total: 308 ms
Wall time: 357 ms


## 📌3. Calcul des indicateurs `Etat de stock (2/2)` `Annexe 2 - Suivi des Stocks`

In [25]:
df_etat_stock_periph = stock_sync_manager.get_table_data(
    query=QUERY_ETAT_STOCK_PERIPH.format(eomonth=eomonth)
)

df_etat_stock_periph = df_etat_stock_periph.loc[
    df_etat_stock_periph.Code_sous_prog.str.contains(programme)
]

df_etat_stock_periph["Code_produit"] = df_etat_stock_periph["Code_produit"].astype(int)

df_etat_stock_periph.head(2)

,Periode,Code_ets,stock_initial,qte_recue,qte_utilisee,perte_ajust,j_rupture,sdu,cmm_esigl,cmm_gest,qte_prop,qte_cmde,qte_approuv,msd,etat_stock,besoin_cmde_urg,besoin_trsf_in,qte_trsf_out,date_report,qte_cmde_mois_prec,etat_stock_mois_prec,id_produit_fk,id_produit_pk,Code_produit,Produit_designation,Unit_rapportage,Categorie_produit,Categorie_du_produit,Code_sous_prog
5628,MARS 2025,50800030,6840,0,2490,0,0,4350,3640,3640.0,10210,10210.0,10210,1.195055,ENTRE PCU et MIN,NaN,NaN,NaN,2025-03-31,NaN,None,96,96,3100014,DOLUTEGRAVIR /LAMIVUDINE/TENOFOVIR 50/300/300 ...,COMPRIME,Produit traceur,MEDICAMENTS ET INTRANTS,PNLS-1
5986,MARS 2025,51600040,810,0,180,0,0,630,60,60.0,0,0.0,0,10.500000,SURSTOCK,NaN,NaN,390.0,2025-03-31,0.0,STOCK DORMANT,97,97,3100016,DOLUTEGRAVIR 10 mg comp. dispersible BTE/90 BT...,COMPRIME,Produit traceur,ARV-ENFANT,PNLS-1


In [26]:
df_plan_approv["Quantité harmonisée (SAGE)"] = df_plan_approv["Quantite"]*df_plan_approv["facteur_de_conversion_qat_sage"]

In [27]:
%%time
df_etat_stock = compute_indicators.annexe_2(
    df_etat_stock,
    df_dmm_curent.copy(),
    df_stock_prog_nat.copy(),
    df_etat_stock_periph.copy(),
    df_stock_detaille.copy(),
    df_receptions.copy(),
    df_plan_approv.copy(),
    date_report,
)

,id_dim_produit_stock_track_fk,stock_theorique_mois_precedent,distribution_effectuee,quantite_recue_stock,quantite_ppi,quantite_prelevee_cq,ajustement_stock,stock_theorique_final_sage,stock_theorique_final_attendu,ecarts,justification_ecarts,diligences,sdu_central_annexe_2,dmm_central_annexe_2,msd_central_annexe_2,statut_central_annexe_2,conso_decentralise_annexe_2,sdu_decentralise_annexe_2,cmm_decentralise_annexe_2,msd_decentralise_annexe_2,statut_decentralise_annexe_2,nombre_de_site_en_rupture_annexe_2,sdu_national_annexe_2,cmm_national_annexe_2,msd_national_annexe_2,statut_national_annexe_2,date_peremption_plus_proche_brute_annexe_2,date_peremption_plus_proche_annexe_2,quantite_correspondante_annexe_2,msd_correspondant_annexe_2,quantite_attendue_annexe_2,msd_attendu_annexe_2,quantite_non_stockee_annexe_2,msd_recu_annexe_2,financement_annexe_2,date_probable_livraison_annexe_2,date_effective_livraison_annexe_2,statut_annexe_2,analyse_risque_commentaires_annexe_2,diligences_central_annexe_2,diligences_peripherique_annexe_2,responsable_annexe_2,dilig_choisie_annexe_2,date_report
0,40,50058.0,4643,0,0,0,None,45532,45415.0,117.0,None,None,45532,6832.000000,"6,7",Bien Stocké,8523,31292,9005,"3,5",Bien Stocké,3,76824,9005,"8,5",Bien Stocké,2027-01-01,2027-01-01,5.0,"0,0",0.0,"0,0",0,"0,0",,NaT,NaT,,None,None,None,None,None,2025-03-01
1,41,34968.0,1391,0,0,0,None,34201,33577.0,624.0,None,None,34201,2427.666667,"14,1",SurStock,2497,11939,2752,"4,3",SurStock,11,46140,2752,"16,8",SurStock,2025-10-01,2025-10-01,3446.0,"1,4",0.0,"0,0",0,"0,0",,NaT,NaT,,None,None,None,None,None,2025-03-01
2,42,0.0,135,0,0,0,None,4145,-135.0,4280.0,None,None,4145,27.000000,"153,5",SurStock,34,94,63,"1,5",Sous-Stock,10,4239,63,"67,3",SurStock,2028-11-01,2028-11-01,4145.0,"153,5",0.0,"0,0",0,"0,0",,NaT,NaT,,None,None,None,None,None,2025-03-01


CPU times: user 2.52 s, sys: 11.3 ms, total: 2.53 s
Wall time: 2.53 s


# V.Exportation des données

Avant d’exporter les données, il est essentiel de s’assurer qu’aucune information relative à la période en cours n’est déjà présente dans les différentes tables.  
Si des données existent, une suppression devra être effectuée avant l’export.

## 1. `Etat de Stock`

In [28]:
id_list = list(map(int, df_etat_stock.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track
WHERE date_report = '{date_report}' AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, list(id_list))
stock_sync_manager.conn.commit()

In [29]:
assert df_etat_stock.date_report.unique().shape[0] == 1

stock_sync_manager.insert_dataframe_to_table(df_etat_stock, "stock_track")

'Insertion de 280 enrégistrements réussie'

## 2. `Distributions`

### 2.1.`Distributions mois courant` 

In [30]:
id_list = list(map(int, df_dmm_curent.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_dmm
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [31]:
assert df_dmm_curent.date_report.unique().shape[0] == 1

stock_sync_manager.insert_dataframe_to_table(df_dmm_curent, "stock_track_dmm")

'Insertion de 280 enrégistrements réussie'

### 2.2.`Distributions hitoriques considérées pour le mois en cours`

In [32]:
id_list = list(map(int, df_dmm_histo.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_dmm_histo
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [33]:
assert df_dmm_histo.date_report.unique().shape[0] == 1

stock_sync_manager.insert_dataframe_to_table(df_dmm_histo, "stock_track_dmm_histo")

'Insertion de 1680 enrégistrements réussie'

## 3.`Consommations`

### 3.1. `Consommations mois courant`

In [34]:
id_list = list(map(int, df_cmm_curent.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_cmm
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [35]:
assert df_cmm_curent.date_report.unique().shape[0] == 1

stock_sync_manager.insert_dataframe_to_table(df_cmm_curent, "stock_track_cmm")

'Insertion de 280 enrégistrements réussie'

### 3.2.`Consommations hitoriques considérées pour le mois en cours`

In [36]:
id_list = list(map(int, df_cmm_histo.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_cmm_histo
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [37]:
assert df_dmm_histo.date_report.unique().shape[0] == 1

stock_sync_manager.insert_dataframe_to_table(df_cmm_histo, "stock_track_cmm_histo")

'Insertion de 1680 enrégistrements réussie'

## 4.`Stock détaillé`

In [38]:
dim_produit = pd.read_sql(
    f"SELECT * FROM {schema_name}.dim_produit_stock_track where programme='{programme}'",
    stock_sync_manager.civ_engine,
)

In [39]:
code_col = [col for col in df_stock_detaille.columns if "CODE" in str(col).upper()][0]

df_stock_detaille.rename(
    columns={
        code_col: "code_produit",
        "Désignation": "designation_produit",
        "Emplacement": "emplacement",
        "Date limite de consommation": "date_limite_consommation",
        "Numéro Lot": "numero_lot",
        "Sous lot (Programme)": "sous_lot_programme",
        "Qté \nPhysique": "qte_physique",
        "Qté \nlivrable": "qte_livrable",
        "Unit": "unit",
    },
    inplace=True,
)
df_stock_detaille = df_stock_detaille[
    [
        "code_produit",
        "designation_produit",
        "emplacement",
        "date_limite_consommation",
        "numero_lot",
        "sous_lot_programme",
        "qte_physique",
        "qte_livrable",
        "unit",
    ]
]

df_stock_detaille = df_stock_detaille.loc[
    df_stock_detaille.code_produit.isin(dim_produit.code_produit)
]
df_stock_detaille.head(1)

,code_produit,designation_produit,emplacement,date_limite_consommation,numero_lot,sous_lot_programme,qte_physique,qte_livrable,unit
0,4040095,COBAS C111 SAMPLE CUPS sach/250 (C 111/C 311),01A01A01,2099-01-01,NEANT,PNLS,1650,0,SACH


In [40]:
df_stock_detaille = (
    df_stock_detaille.groupby(["code_produit", "date_limite_consommation"])[
        ["qte_physique", "qte_livrable"]
    ]
    .sum(min_count=1)
    .reset_index()
)

df_stock_detaille = (
    dim_produit[["id_dim_produit_stock_track_pk", "code_produit"]]
    .merge(df_stock_detaille, on="code_produit")
    .rename(columns={"id_dim_produit_stock_track_pk": "id_dim_produit_stock_track_fk"})
    .drop(columns=["code_produit"])
)

df_stock_detaille["date_report"] = pd.to_datetime(date_report)

In [41]:
id_list = list(map(int, df_stock_detaille.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_detaille
WHERE date_report = '{date_report}' AND id_dim_produit_stock_track_fk IN ({placeholders});
"""

stock_sync_manager.civ_cursor.execute(query, list(id_list))
stock_sync_manager.conn.commit()

In [42]:
stock_sync_manager.insert_dataframe_to_table(df_stock_detaille, "stock_track_detaille")

del df_stock_detaille

## 5. `Etat de stock de la NPSP`

In [43]:
query = f"""
DELETE FROM {schema_name}.stock_track_npsp
WHERE date_report = '{date_report}' and programme='{programme}';
"""

stock_sync_manager.civ_cursor.execute(query)
stock_sync_manager.conn.commit()

In [44]:
stock_sync_manager.insert_dataframe_to_table(
    df_etat_stock_npsp, table_name="stock_track_npsp", schema_name="suivi_stock"
)

'Insertion de 287 enrégistrements réussie'

## 6.`Prévision`
<div class="alert alert-block alert-warning">
Avant de procéder au calcul des éléments de prévision, il est essentiel de s’assurer que les informations produits sont à jour.
Cette actualisation doit être effectuée à l’aide du fichier de mapping fourni en paramètre du pipeline.
Ce fichier joue un rôle clé : il établit la correspondance entre les différentes sources de données, garantissant ainsi la cohérence et la fiabilité des informations utilisées pour les prévisions.

</div>

In [45]:
stock_sync_manager.synchronize_product_metadata(df_plan_approv.copy(), programme)

'Aucune mise à jour des données à effectuer sur la table dim_produit'

In [46]:
df_plan_approv['Date updated'] = df_plan_approv['DATE'].apply(lambda x: x.strftime("%b-%Y"))

In [47]:
df_prevision = compute_indicators.prevision.get_prevision_current_month(
    df_plan_approv.copy(),
    date_report,
    programme,
    stock_sync_manager.civ_engine,
    schema_name="suivi_stock",
)

,id_dim_produit_stock_track_fk,code_produit,ancien_code,stock_central,dmm_central,stock_national,cmm_national,stock_prev_central,stock_prev_national,period_prev,date_report
0,66,3050188,AR01321,357.0,367.16666,2081.0,470.0,1,4,2025-03-01,2025-03-01
1,66,3050188,AR01321,NaN,NaN,NaN,NaN,0,3,2025-04-01,2025-03-01
2,66,3050188,AR01321,NaN,NaN,NaN,NaN,8,8,2025-05-01,2025-03-01


In [48]:
id_list = list(map(int, df_prevision.id_dim_produit_stock_track_fk.unique()))
placeholders = ", ".join(["%s"] * len(id_list))

query = f"""
DELETE FROM {schema_name}.stock_track_prevision
WHERE date_report = %s AND id_dim_produit_stock_track_fk IN ({placeholders});
"""
stock_sync_manager.civ_cursor.execute(query, [date_report] + list(id_list))
stock_sync_manager.conn.commit()

In [49]:
stock_sync_manager.insert_dataframe_to_table(
    df_prevision.drop(columns=["code_produit", "ancien_code"]), "stock_track_prevision"
)

'Insertion de 3640 enrégistrements réussie'

## 7.`Plan d'approvisionnement`

In [50]:
query = f"""
DELETE FROM {schema_name}.plan_approv
WHERE date_report = '{date_report}' and programme='{programme}'
"""
stock_sync_manager.civ_cursor.execute(query)
stock_sync_manager.conn.commit()

In [51]:
df_pa = df_plan_approv.rename(
    columns={
        "Standard product code": "standard_product_code",
        "ID de produit QAT": "id_produit_qat",
        "Produits": "designation",
        "ID de l`envoi QAT": "id_envoi_qat",
        "Agent d`approvisionnement": "centrale_achat",
        "Source de financement": "source_financement",
        "Status": "status",
        "Quantite": "quantite",
        "facteur_de_conversion_qat_sage": "facteur_conversion_qat_vers_sage",
        "Quantité harmonisée (SAGE)": "quantite_harmonisee_sage",
        "DATE": "date",
        "Cout des Produits": "cout_produits",
        "Couts du fret": "cout_fret",
        "Couts totaux": "cout_total",
    },
)

df_pa = df_pa[
    ["standard_product_code", "id_produit_qat",
     "designation", "id_envoi_qat",
     "centrale_achat", "source_financement",
     "status", "quantite", "facteur_conversion_qat_vers_sage",
     "quantite_harmonisee_sage", "date",
     "cout_produits", "cout_fret", "cout_total",
    ]
]
df_pa["date_report"] = pd.to_datetime(date_report)
df_pa["programme"] = programme
df_pa["standard_product_code"] = df_pa["standard_product_code"].fillna(0)

df_pa.head(2)

,standard_product_code,id_produit_qat,designation,id_envoi_qat,centrale_achat,source_financement,status,quantite,facteur_conversion_qat_vers_sage,quantite_harmonisee_sage,date,cout_produits,cout_fret,cout_total,date_report,programme
0,3120042.0,3067,Female Condom (Nitrile) Lubricated 17 cm 100...,173896,PPM-GF,GFATM,Planifié,200.000,1.0,200.000,2025-12-31,100000.00,10000.0,110000.00,2025-03-01,PNLS
1,3120044.0,4182,Male Condom (Latex) Lubricated No Logo 53 mm...,173899,PPM-GF,GFATM,Soumis,4147.584,1.0,4147.584,2025-07-31,401776.46,40181.7,441958.16,2025-03-01,PNLS


In [52]:
stock_sync_manager.insert_dataframe_to_table(df_pa, table_name="plan_approv", schema_name="suivi_stock")

'Insertion de 527 enrégistrements réussie'

In [53]:
del df_pa

# VI.📄Génération du fichier pour le mois spécifique

## 1. Initialisation des paramètres utile

In [54]:
date_report_format = date_report
date_report = pd.to_datetime(date_report, format='%Y-%m-%d').strftime('%d/%m/%Y')
f_month = gstf.get_current_variable(date_report)[1].replace(' ', '-')

## 2. Importation des classeurs de bases

In [55]:
%%time
# WorkBook Template de base
wb_template = pyxl.load_workbook(
    filename="/home/jovyan/workspace/Fichier Suivi de Stock/code/pipelines/generate_stock_tracking_file/Template Fichier Suivi de Stock/Fichier Suivi de Stock Template.xlsx"
)

# WorkBook Etats de stock mensuels
wb_etat_stock = pyxl.load_workbook(filename=fp_etat_mensuel)

# WorkBook du Rapport Feedback
wb_fbr = pyxl.load_workbook(
    filename=f"/home/jovyan/workspace/Rapport Feedback/code/pipelines/rapport feedback genere/{date_report_format[:4]}/RapportFeedBack-{unidecode(f_month)}.xlsx"
)

CPU times: user 6min 20s, sys: 10.6 s, total: 6min 30s
Wall time: 6min 32s


## 3. Mise à jour des données en se basant sur la source `Etat de Stock Mensuel`

In [56]:
%%time
wb_template = gstf.update_sheets_etat_mensuel(
    wb_etat_stock, wb_template, programme, date_report
)

CPU times: user 1min 28s, sys: 107 ms, total: 1min 28s
Wall time: 1min 28s


## 4. Mise à jour des données en se basant sur la source `Rapport Feedback`

Pour le faire on va se baser sur les inputs au préalable du fichier RapportFeebback généré et présent dans les fichiers d'OH pour le mois courant

In [57]:
df_etat_stock = pd.read_excel(
    f"/home/jovyan/workspace/Rapport Feedback/code/pipelines/rapport feedback genere/{date_report_format[:4]}/RapportFeedBack-{unidecode(f_month)}.xlsx",
    sheet_name="ETAT DU STOCK",
)

df_etat_stock.head(2)

,Code_Pro,CODE,PROGRAMME,SOUS-PROGRAMME,PERIODE,REGION,DISTRICT,CODE ETS,STRUCTURE,TYPE DE STRUCTURE,CATEGORIE PRODUIT,PRODUIT,UNITE DE RAPPORTAGE,STOCK INITIAL,QUANTITE RECUE,QUANTITE UTILISEE,PERTES ET AJUSTEMENT,JOURS DE RUPTURE,SDU,CMM ESIGL,CMM gestionnaire,QUANTITE PROPOSEE,QUANTITE COMMANDEE,QUANTITE APPROUVEE,MSD,ETAT DU STOCK,BESOIN COMMANDE URGENTE,BESOIN TRANSFERT IN,QUANTITE A TRANSFERER OUT,CATEGORIE_DU_PRODUIT,Commandes du mois précédent,Etat du stock du mois précédent
0,3050350_PNLT,3050350,PNLT,PNLT-SENSIBLE MEDICAMENTS ET INTRANTS,JANVIER MARS 2025,IFFOU,DAOUKRO,62600030,CENTRE ANTITUBERCULEUX DAOUKRO,FORMATION SANITAIRE,Produit traceur,RH 150 (RIFAmpICINE150 - ISONIAZIDE75) comp. B...,COMPRIME,9316,9408,8992,-2399,0,7333,1007,3021.0,0,10793,10793,2.427342,ENTRE PCU et MIN,NaN,1730.0,NaN,PRODUITS PNLT ADULTE,NaN,NaN
1,3050356_PNLT,3050356,PNLT,PNLT-SENSIBLE MEDICAMENTS ET INTRANTS,JANVIER MARS 2025,IFFOU,DAOUKRO,62600030,CENTRE ANTITUBERCULEUX DAOUKRO,FORMATION SANITAIRE,Produit traceur,RHZE (RIFAMP 150 - ISON 75 - PYRAZ 400 - ETHAM...,COMPRIME,8862,0,5251,-2016,0,1595,591,1774.0,1951,9049,9049,0.899098,EN BAS DU PCU,9049.0,3727.0,NaN,PRODUITS PNLT ADULTE,NaN,NaN


In [58]:
%%time

wb_template = gstf.update_sheet_etat_stock(wb_template, df_etat_stock.loc[df_etat_stock.PROGRAMME==programme].copy())

wb_template = gstf.update_sheet_stock_region(wb_template, wb_fbr, programme)

CPU times: user 39.5 s, sys: 189 ms, total: 39.7 s
Wall time: 39.7 s


## 4. Mise à jour des informations feuille `Annexe 1 - Consolidation`

In [59]:
wb_template = gstf.update_sheet_annexe_1(
    wb_template, programme, schema_name, stock_sync_manager.civ_engine, date_report
)

## 5. Mise à jour des données du `Plan d'approvisionnement`

In [60]:
df_pa = df_plan_approv.rename(
    columns={
        "Agent d`approvisionnement": "Centrale d'achat",
        "Source de financement": "Source Financement",
        "facteur_de_conversion_qat_sage": "Facteur de conversion de QAT vers SAGE",
        "cout_unitaire_moyen_qat": "Coût unitaire moyen (en dollar)",
        "acronym": "Acronym",
    }
)

df_pa["DATE"] = pd.to_datetime(df_pa["DATE"], format='%d-%b-%Y', errors='coerce')

In [61]:
%%time
wb_template = gstf.update_sheet_plan_approv(wb_template, df_pa)

CPU times: user 761 ms, sys: 966 μs, total: 762 ms
Wall time: 761 ms


## 5. Mise à jour des informations  en se basant sur la source `Annexe 2 - Suivi de Stock`

In [62]:
%%time
wb_template = gstf.update_sheet_annexe_2(wb_template, df_pa, date_report)

CPU times: user 1.94 s, sys: 2.05 ms, total: 1.94 s
Wall time: 1.94 s


## 6. Mise à jour des données en se basant sur la source `Prévision`

In [63]:
df_prod = dim_produit[
    ["code_produit", "type_produit", "designation", "designation_acronym"]
]

df_prod["designation"] = df_prod.apply(
    lambda row: row["designation_acronym"]
    if not pd.isna(row["designation_acronym"])
    else row["designation"],
    axis=1,
)
df_prod = df_prod.drop(columns="designation_acronym")

In [64]:
wb_template = gstf.update_sheet_prevision(wb_template, date_report, df_prod)

## 7. Mise à jour de certains éléments de la feuille `Rapport`

In [65]:
ws_rapport = wb_template['Rapport']
ws_rapport['D4'].value = programme
ws_rapport['D6'].value = f_month.split('-')[0].capitalize()
ws_rapport['E6'].value = date_report_format[:4]

# VII. Exportation du Fichier Suivi de Stock vers le `drive` et la `BD`

In [66]:
%%time
# Sauvegarde du fichier dans un repertoire courant
dest_file = (
    Path(workspace.files_path)
    / f"Fichier Suivi de Stock/code/pipelines/fichier suivi de stock genere/{programme}/{date_report_format[:4]}"
)

dest_file.mkdir(exist_ok=True, parents=True)

dest_file = dest_file / f"Fichier Suivi de Stock {programme}-{f_month}.xlsx"

wb_template.save(dest_file)

CPU times: user 7.74 s, sys: 213 ms, total: 7.96 s
Wall time: 8.65 s


In [67]:
download_url = ggdrive.upload_and_return_link(
    dest_file.as_posix(),
    date_report_format,
)

Aucun file trouvé avec ce nom.


In [68]:
df_download_url = pd.DataFrame(
    data=[
        {
            "programme": programme,
            "download_url": download_url,
            "date_report": date_report_format,
        }
    ]
)

df_download_url.date_report = df_download_url.date_report.apply(
    lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d")
)

In [69]:
stock_sync_manager.synchronize_table_data(
    df_download_url,
    table_name="share_link",
    merge_keys=["programme", "date_report"],
    programme=programme,
)

Insertion de 1 enrégistrements réussie


'Aucune mise à jour des données à effectuer'